In [69]:
import random
from copy import deepcopy
def initSudoku(sudoku):
    sudokuPuzzle = deepcopy(sudoku)
    #for row in range(len(sudokuPuzzle)):
    #    for col in range(len(sudokuPuzzle[row])):
    #        if(sudokuPuzzle[row][col] == '.'):
    #            rand = random.randint(1, 9)
    #            sudokuPuzzle[row][col] = rand

    for r in range(3):
        for c in range(3):

            valueChecker = []
            for i in range(9):
                valueChecker.append(0)

            rowStartPositions = [0, 3, 6]
            colStartPositions = [0, 3, 6]
            #First 3x3 row and col
            row = rowStartPositions[r]
            col = colStartPositions[c]

            countRow = 0
            countCol = 0
            #Scan the 3x3 first and see what is already there
            while((countRow <= 2) and (countCol <= 2)):
                if(sudokuPuzzle[row + countRow][col + countCol] != '.'):
                    valueChecker[sudokuPuzzle[row + countRow][col + countCol] - 1] = sudokuPuzzle[row + countRow][col + countCol]

                countCol = countCol + 1
                if(countCol > 2):
                    countRow = countRow + 1
                    countCol = 0

            
            #print()
            #print(valueChecker)
            #Next assign random values from 1-9 in the 3x3 but it can only appear once in the 3x3
            #First 3x3 row and col
            row = rowStartPositions[r]
            col = colStartPositions[c]

            countRow = 0
            countCol = 0
            #Scan the 3x3 first and see what is already there
            while((countRow <= 2) and (countCol <= 2)):
                if(sudokuPuzzle[row + countRow][col + countCol] == '.'):
                    rand = random.randint(1, 9)
                    while(rand in valueChecker):
                        rand = random.randint(1, 9)
                    sudokuPuzzle[row + countRow][col + countCol] = rand
                    valueChecker[rand - 1] = rand

                countCol = countCol + 1
                if(countCol > 2):
                    countRow = countRow + 1
                    countCol = 0



    
    return sudokuPuzzle



In [144]:
def getPenalty(sudoku):
    penalty = 0
    OriRowcounter = []
    OriColcounter = []
    for i in range(len(sudoku)):
        OriRowcounter.append(0)
        OriColcounter.append(0)

    #Get row penalty
    for row in range(len(sudoku)):

        rowCount = OriRowcounter.copy()
        for col in range(len(sudoku[row])):
            if(rowCount[sudoku[row][col] - 1] == 0):
                rowCount[sudoku[row][col] - 1] = 1

        rowSum = sum(rowCount)
        #print("Row: " + str(row) + ", penalty: " + str(rowSum))
        penalty = penalty + (9- rowSum)

    
    for col in range(len(sudoku[0])):
        colCount = OriColcounter.copy()
        for row in range(len(sudoku)):
            if(colCount[sudoku[row][col] - 1] == 0):
                colCount[sudoku[row][col] - 1] = 1
        
        colSum = sum(colCount)
        #print("col: " + str(col) + ", penalty: " + str(colSum))
        penalty = penalty + (9- colSum)

    return penalty
        

In [47]:
def getFitness(sudoku):
    penalty = getPenalty(sudoku)
    fitness = 1 / (1 + penalty)
    return fitness

In [164]:
def getProbability(fitness_scores):
    probabities = []
    sumFitness = sum(fitness_scores)
    for i in range(len(fitness_scores)):
        probabities.append(fitness_scores[i] / sumFitness)

    return probabities



In [84]:
def getCellPosition(randJ):
    row = int(randJ/9)
    col = randJ % 9
    #print("row: " + str(row))
    #print("col: " + str(col))
    return row, col

In [154]:
def checkStartSquare(originalPuzzle, rowJ, colJ, vij):
    
    if(originalPuzzle[rowJ][colJ] != '.'):
        #print("Already a start square. row:" + str(rowJ) + ", col: " + str(colJ))
        return True
    
    if(vij == 0):
        return False
    
    #check the col if vij is a start square
    for i in range(len(originalPuzzle)):
        if(originalPuzzle[i][colJ] == vij):
            #print("Found in col. row:" + str(i) + ", col: " + str(colJ))
            return True


    #check the col if vij is a start square
    for j in range(len(originalPuzzle[rowJ])):
        if(originalPuzzle[rowJ][j] == vij):
            #print("Found in row. row:" + str(rowJ) + ", col: " + str(j))
            return True


    #find the 3x3 block
    rowStartPositions = [0, 3, 6]
    colStartPositions = [0, 3, 6]
    #find starting row of 3x3 block
    rowStart = 0
    if( rowJ >= rowStartPositions[2]):
        rowStart = rowStartPositions[2]
    elif((rowJ >= rowStartPositions[1]) and (rowJ < rowStartPositions[2])):
        rowStart = rowStartPositions[1]
    else:
        rowStart = rowStartPositions[0]

    #find starting col of 3x3 block
    colStart = 0
    if( colJ >= colStartPositions[2]):
        colStart = colStartPositions[2]
    elif((colJ >= colStartPositions[1]) and (colJ < colStartPositions[2])):
        colStart = colStartPositions[1]
    else:
        colStart = colStartPositions[0]

    countRow = 0
    countCol = 0

    #Loop through the 3x3 square
    while((countRow <= 2) and (countCol <= 2)):
        
        if(originalPuzzle[rowStart + countRow][colStart + countCol] == vij):
            #print("Found in 3x3 square. row:" + str(rowStart + countRow) + ", col: " + str(colStart + countCol))
            return True
        countCol = countCol + 1
        if(countCol > 2):
            countRow = countRow + 1
            countCol = 0

    return False

    
        
    

In [160]:
from copy import deepcopy
def swapOperation(puzzle, rowJ, colJ, vij):
    newPuzzle = deepcopy(puzzle)
    temp = newPuzzle[rowJ][colJ]

    #find the 3x3 block
    rowStartPositions = [0, 3, 6]
    colStartPositions = [0, 3, 6]
    #find starting row of 3x3 block
    rowStart = 0
    if( rowJ >= rowStartPositions[2]):
        rowStart = rowStartPositions[2]
    elif((rowJ >= rowStartPositions[1]) and (rowJ < rowStartPositions[2])):
        rowStart = rowStartPositions[1]
    else:
        rowStart = rowStartPositions[0]

    #find starting col of 3x3 block
    colStart = 0
    if( colJ >= colStartPositions[2]):
        colStart = colStartPositions[2]
    elif((colJ >= colStartPositions[1]) and (colJ < colStartPositions[2])):
        colStart = colStartPositions[1]
    else:
        colStart = colStartPositions[0]

    countRow = 0
    countCol = 0

    #Loop through the 3x3 square
    while((countRow <= 2) and (countCol <= 2)):
        
        #print("newPuzzle[countRow][countCol]: " + str(newPuzzle[rowStart + countRow][colStart + countCol] ))
        if(newPuzzle[rowStart + countRow][colStart + countCol] == vij):
            #print("Found at Row: " + str(rowStart + countRow) + ", col: " + str(colStart + countCol))
            newPuzzle[rowStart + countRow][colStart + countCol] = temp
        
        countCol = countCol + 1
        if(countCol > 2):
            countRow = countRow + 1
            countCol = 0
    
    newPuzzle[rowJ][colJ] = vij

    return newPuzzle

In [158]:
puzzle = [[4, 5, 2, 2, 6, 3, 7, 3, 1],
        [6, 8, 3, 1, 7, 8, 4, 9, 6],
        [1, 9, 7, 9, 5, 4, 5, 2, 8],
        [8, 2, 1, 1, 5, 8, 1, 4, 5],
        [6, 3, 4, 6, 4, 2, 9, 6, 7],
        [7, 5, 9, 7, 9, 3, 3, 2, 8],
        [8, 2, 9, 3, 4, 6, 1, 7, 4],
        [6, 4, 5, 7, 5, 2, 2, 3, 6],
        [7, 1, 3, 9, 1, 8, 5, 9, 8]]

rowJ = 5
colJ = 5
vij = 1
new_puzzle = swapOperation(puzzle, rowJ, colJ, vij)

for k in range(len(puzzle)):
        print(puzzle[k])

print()
for k in range(len(new_puzzle)):
        print(new_puzzle[k])




countRow: 0
countCol: 0
newPuzzle[countRow][countCol]: 1
Found at Row: 3, col: 3
newPuzzle[countRow][countCol]: 5
newPuzzle[countRow][countCol]: 8
newPuzzle[countRow][countCol]: 6
newPuzzle[countRow][countCol]: 4
newPuzzle[countRow][countCol]: 2
newPuzzle[countRow][countCol]: 7
newPuzzle[countRow][countCol]: 9
newPuzzle[countRow][countCol]: 3
[4, 5, 2, 2, 6, 3, 7, 3, 1]
[6, 8, 3, 1, 7, 8, 4, 9, 6]
[1, 9, 7, 9, 5, 4, 5, 2, 8]
[8, 2, 1, 1, 5, 8, 1, 4, 5]
[6, 3, 4, 6, 4, 2, 9, 6, 7]
[7, 5, 9, 7, 9, 3, 3, 2, 8]
[8, 2, 9, 3, 4, 6, 1, 7, 4]
[6, 4, 5, 7, 5, 2, 2, 3, 6]
[7, 1, 3, 9, 1, 8, 5, 9, 8]

[4, 5, 2, 2, 6, 3, 7, 3, 1]
[6, 8, 3, 1, 7, 8, 4, 9, 6]
[1, 9, 7, 9, 5, 4, 5, 2, 8]
[8, 2, 1, 3, 5, 8, 1, 4, 5]
[6, 3, 4, 6, 4, 2, 9, 6, 7]
[7, 5, 9, 7, 9, 1, 3, 2, 8]
[8, 2, 9, 3, 4, 6, 1, 7, 4]
[6, 4, 5, 7, 5, 2, 2, 3, 6]
[7, 1, 3, 9, 1, 8, 5, 9, 8]


In [153]:
from copy import deepcopy
sudokuDataBase = [['.','.','.',2,6,'.',7,'.',1], 
                    [6,8,'.','.',7,'.','.',9,'.'],
                    [1,9,'.','.','.',4,5,'.','.'],
                    [8,2,'.',1,'.','.','.',4,'.'],
                    ['.','.',4,6,'.',2,9,'.','.'],
                    ['.',5,'.','.','.',3,'.',2,8],
                    ['.','.',9,3,'.','.','.',7,4],
                    ['.',4,'.','.',5,'.','.',3,6],
                    [7,'.',3,'.',1,8,'.','.','.']]

for k in range(len(sudokuDataBase)):
        print(sudokuDataBase[k])


original = deepcopy(sudokuDataBase)

#filledPuzzle = initSudoku(sudokuDataBase.copy())


#print()
#for k in range(len(filledPuzzle)):
#        print(filledPuzzle[k])

print()
#penalty = getPenalty(filledPuzzle)
#print("penalty: " + str(penalty))
#fitness = getFitness(filledPuzzle)
#print("fitness: " + str(fitness))

#rowJ, colJ = getCellPosition(38)
#print(filledPuzzle[rowJ][colJ])

rowJ = 8
colJ = 7
vij = 6

checkStartSquare(original, rowJ, colJ, vij)


['.', '.', '.', 2, 6, '.', 7, '.', 1]
[6, 8, '.', '.', 7, '.', '.', 9, '.']
[1, 9, '.', '.', '.', 4, 5, '.', '.']
[8, 2, '.', 1, '.', '.', '.', 4, '.']
['.', '.', 4, 6, '.', 2, 9, '.', '.']
['.', 5, '.', '.', '.', 3, '.', 2, 8]
['.', '.', 9, 3, '.', '.', '.', 7, 4]
['.', 4, '.', '.', 5, '.', '.', 3, 6]
[7, '.', 3, '.', 1, 8, '.', '.', '.']

Found in 3x3 square. row:7, col: 8


True

In [196]:
from copy import deepcopy

def sudokuABC(sudokuDataBase, e, o, c):
    s = int(0.1*e) #number of scout bees

    #initilaize employee bees food sources

    food_sources = []

    for i in range(e):
        food_sources.append(initSudoku(deepcopy(sudokuDataBase)))


    #initilaize fitness for the food sources
    fitness = []
    for i in range(e):
        fitness.append(getFitness(food_sources[i]))

    #print(fitness)

    best_fitness = 0
    best_food_source = []

    solution_found = False
    cCounter = 0
    #Start loop here with termination conditions
    while((cCounter <= c) and (solution_found == False)):

        if((cCounter % 100) == 0):
            print("cCounter: " + str(cCounter))


        #employee bee phase
        for i in range(e):
            #Choose random neighbor
            randNeighor = random.randint(0, e-1)
            while(randNeighor == i):
                randNeighor = random.randint(0, e-1)

            randJ = random.randint(0, 80) #Random Cell

            rowJ, colJ = getCellPosition(randJ)
            #Check if rowJ and colJ is a start square
            isStartSquare = True
            while(isStartSquare == True):
                randJ = random.randint(0, 80) #Random Cell
                rowJ, colJ = getCellPosition(randJ)

                vij = food_sources[i][rowJ][colJ] + random.random() * abs(food_sources[i][rowJ][colJ] - food_sources[randNeighor][rowJ][colJ])
                vij = int(vij)
                if(vij > 9):
                    vij = (vij % 9) + 1

                isStartSquare = checkStartSquare(sudokuDataBase, rowJ, colJ, vij)
            
            #Debug print
            #for k in range(len(food_sources[i])):
            #    print(food_sources[i][k])

            #Do a swap operation to replace vij in the new food source 
            new_food_source = swapOperation(food_sources[i], rowJ, colJ, vij)

            #Debug print
            #print()
            #for k in range(len(new_food_source)):
            #    print(new_food_source[k])

            #Find the fitness of the new food source and compare with the current food source fitness
            new_fitness = getFitness(new_food_source)
            if(new_fitness > fitness[i]):
                #Better food source so replace the food source and fitness
                #print("Better fitness: " + str(new_fitness))
                #print("old fitness: " + str(fitness[i]))
                food_sources[i] = new_food_source
                fitness[i] = new_fitness
                #print("New fitness: " + str(fitness[i]))

            #print("row: " + str(rowJ) + ", col: " + str(colJ) + ", vij: " + str(vij))


        #Onlooker bee phase

        proability_scores = getProbability(fitness)
        onlookerCounter = 0
        employeeCounter = 0
        #print("proability_scores" + str(proability_scores))
        while(onlookerCounter <= o):
            proability_scores = getProbability(fitness)
            onlookerRand = random.random()

            if(onlookerRand < proability_scores[employeeCounter]):
                #print("onlookerRand" + str(onlookerRand))
                #print("proability_scores[employeeCounter]" + str(proability_scores[employeeCounter]))
                onlookerCounter = onlookerCounter + 1

                #Choose random neighbor
                randNeighor = random.randint(0, e-1)
                while(randNeighor == employeeCounter):
                    randNeighor = random.randint(0, e-1)

                randJ = random.randint(0, 80) #Random Cell

                rowJ, colJ = getCellPosition(randJ)
                #Check if rowJ and colJ is a start square
                isStartSquare = True
                while(isStartSquare == True):
                    randJ = random.randint(0, 80) #Random Cell
                    rowJ, colJ = getCellPosition(randJ)

                    vij = food_sources[employeeCounter][rowJ][colJ] + random.random() * abs(food_sources[employeeCounter][rowJ][colJ] - food_sources[randNeighor][rowJ][colJ])
                    vij = int(vij)
                    if(vij > 9):
                        vij = (vij % 9) + 1

                    isStartSquare = checkStartSquare(sudokuDataBase, rowJ, colJ, vij)

                #Do a swap operation to replace vij in the new food source 
                new_food_source = swapOperation(food_sources[employeeCounter], rowJ, colJ, vij)

                #Find the fitness of the new food source and compare with the current food source fitness
                new_fitness = getFitness(new_food_source)
                if(new_fitness > fitness[employeeCounter]):
                    #Better food source so replace the food source and fitness
                    food_sources[employeeCounter] = new_food_source
                    fitness[employeeCounter] = new_fitness
                    #print("Replaced")


            employeeCounter = employeeCounter + 1
            if(employeeCounter == len(food_sources)):
                employeeCounter = 0

        #Scout bee phase
        worst_fitness = 1
        worst_food_source_num = 0
        for j in range(s):
            for i in range(e):
                if(worst_fitness > fitness[i]):
                    worst_fitness = fitness[i]
                    worst_food_source_num = i
            
            
            #Scout bee generating a new food source
            scout_food_source = initSudoku(deepcopy(sudokuDataBase))
            scout_fitness = getFitness(scout_food_source)

            #Replace this worst food source if fitness is better
            if(scout_fitness > worst_fitness):
                food_sources[worst_food_source_num] = scout_food_source
                fitness[worst_food_source_num] = scout_fitness
            



        #Memorize best solution
        for i in range(e):
                if(best_fitness < fitness[i]):
                    best_fitness = fitness[i]
                    best_food_source = food_sources[i]


        if(best_fitness == 1):
            solution_found = True
        cCounter = cCounter + 1


    #Debug print best food source
    print("Best food source")
    for i in range(len(best_food_source)):
        print(best_food_source[i])


    print("Best fitness: " + str(best_fitness))

        





In [197]:
sudokuDataBase = [['.','.','.',2,6,'.',7,'.',1], 
                    [6,8,'.','.',7,'.','.',9,'.'],
                    [1,9,'.','.','.',4,5,'.','.'],
                    [8,2,'.',1,'.','.','.',4,'.'],
                    ['.','.',4,6,'.',2,9,'.','.'],
                    ['.',5,'.','.','.',3,'.',2,8],
                    ['.','.',9,3,'.','.','.',7,4],
                    ['.',4,'.','.',5,'.','.',3,6],
                    [7,'.',3,'.',1,8,'.','.','.']]


e = 100 #number of employee bees
o = 200 #number of onlooker bees
c = 100000  #number of cycles bees
s = int(0.1*e) #number of scout bees

sudokuABC(sudokuDataBase, e, o, c)

cCounter: 0
cCounter: 100
cCounter: 200
cCounter: 300
cCounter: 400
cCounter: 500
cCounter: 600
cCounter: 700
cCounter: 800
cCounter: 900
cCounter: 1000
cCounter: 1100
cCounter: 1200
cCounter: 1300
cCounter: 1400
cCounter: 1500
cCounter: 1600
cCounter: 1700
cCounter: 1800
cCounter: 1900
cCounter: 2000
cCounter: 2100
cCounter: 2200
cCounter: 2300
cCounter: 2400
cCounter: 2500
cCounter: 2600
cCounter: 2700
cCounter: 2800
cCounter: 2900
cCounter: 3000
cCounter: 3100
cCounter: 3200
cCounter: 3300
cCounter: 3400
cCounter: 3500
cCounter: 3600
cCounter: 3700
cCounter: 3800
cCounter: 3900
cCounter: 4000
cCounter: 4100
cCounter: 4200
cCounter: 4300
cCounter: 4400
cCounter: 4500
cCounter: 4600
cCounter: 4700
cCounter: 4800
cCounter: 4900
cCounter: 5000
cCounter: 5100
cCounter: 5200
cCounter: 5300
cCounter: 5400
cCounter: 5500
cCounter: 5600
cCounter: 5700
cCounter: 5800
cCounter: 5900
cCounter: 6000
cCounter: 6100
cCounter: 6200
cCounter: 6300
cCounter: 6400
cCounter: 6500
Best food source
[4, 3